<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); font-family: 'Times New Roman', serif;text-align: justify;font-size: 16px"> 
    
    
### Description of the function

The `binomial_option_pricing` function described in the code implements the binomial option pricing model to calculate the theoretical price of a financial option, either a call or a put, based on various input parameters. It constructs a binomial tree to represent the potential movements of the underlying asset's price over time, and works backward from the expiry date to compute the option's value. The function also accommodates both European and American option styles. For European options, the option can only be exercised at expiration, while for American options, early exercise is allowed at any point before expiry, and this flexibility is incorporated into the pricing model.
    
The function first defines the input parameters and calculates the key variables: the time interval, up factor \( u \), down factor \( d \), risk-neutral probability \( p \), and the discount factor. It then constructs the stock price tree by iterating through each time step and calculating the asset price at each node. For each node at time step \( i \), the price is determined as:
$$ \text{stock price}_{i,j} = \text{spot price} \cdot u^{(i-j)} \cdot d^j $$
where $i$ is the time step interval and $j$ is the number of down moves.
    
Next, the function creates an `option_tree`matrix to store the option values at each node. It computes the option's payoff at the final time step, which corresponds to the expiration date. This final payoff is stored at the last column (corresponding to expiry) for each node.
The function then works backward from the expiry to the present, iterating through each time step. For each node, the option value is calculated by considering the option value at the next time step. For a **European option**, this is done by computing the discounted expected value of the option at the next step, using the risk-neutral probabilities \( p \) and the discount factor. 
For an **American option**, the function also takes into account the possibility of early exercise. For each node, it compares the value of exercising the option immediately (i.e., the intrinsic value) with the discounted expected value of holding the option. The option value at that node is the maximum of these two values. This ensures that the option holder would choose the most beneficial option: either exercising early or waiting for the option to expire.

By iterating backward through the entire binomial tree, the function eventually calculates the option value at the root node, which corresponds to the present time. This final value is stored in `option_tree[0, 0]`, representing the theoretical and fair price of the option.
    
Finally the function measures the computation time (i.e. the time it takes to perform the calculations) and the number of possible paths in the binomial tree grows exponentially with the number of steps. At each step, the option can either move up or down, leading to $2^{\text{steps}}$ possible paths.
    



In [3]:
import numpy as np
import time


In [16]:
def binomial_option_pricing(
    option_type="call", 
    style="European", 
    strike_price=100, 
    spot_price=105, 
    volatility=0.2, 
    risk_free_rate=0.02, 
    time_to_expiry=1, 
    steps=100
):
    start_time = time.time()  #Start timing the computation
    dt = time_to_expiry / steps  
    u = np.exp(volatility * np.sqrt(dt))  #Upward movement factor
    d = np.exp(-volatility * np.sqrt(dt)) #Downward movement factor
    p = (np.exp(risk_free_rate * dt) - d) / (u - d)  #Risk-neutral probability
    discount = np.exp(-risk_free_rate * dt)  #Discount factor for present value calculations
    
    #Creation of the stock price tree
    stock_tree = np.zeros((steps + 1, steps + 1))  
    for i in range(steps + 1):  #Loop over time steps
        for j in range(i + 1):  #Loop over nodes at each time step
            stock_tree[j, i] = spot_price * (u ** (i - j)) * (d ** j) #Stock price at each node is determined by the number of up/down movements
    
    #Option value tree
    option_tree = np.zeros((steps + 1, steps + 1))
    #Computation of the option payoff at expiration (final step)
    if option_type == "call":
        option_tree[:, steps] = np.maximum(stock_tree[:, steps] - strike_price, 0)
    else:  #Put option
        option_tree[:, steps] = np.maximum(strike_price - stock_tree[:, steps], 0)
    
    #Work backward through the tree to compute option values at earlier nodes
    for i in range(steps - 1, -1, -1):  #Loop backward through time steps
        for j in range(i + 1):  #Loop through all nodes at step i
            if style == "European":
                option_tree[j, i] = discount * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])
            else:  #American option:
                early_exercise = stock_tree[j, i] - strike_price if option_type == "call" else strike_price - stock_tree[j, i]
                option_tree[j, i] = max(early_exercise,discount * (p * option_tree[j, i + 1] + (1 - p) * option_tree[j + 1, i + 1])) 
    
    end_time = time.time()  #End timing the computation
    computation_time = end_time - start_time 
    
    #Number of paths considered in the binomial model (each step has 2 possible moves: up or down)
    paths_considered = 2 ** steps  
    
    return option_tree[0, 0], computation_time, paths_considered



In [17]:
price, time_taken, paths = binomial_option_pricing()
print(f"Option Price: {price}, Computation Time: {time_taken} sec, Paths Considered: {paths}")

Option Price: 12.054131439435547, Computation Time: 0.007497072219848633 sec, Paths Considered: 1267650600228229401496703205376
